# Purpose: To build out the skeleton Pseudo Code for this project

__For now__: Moving my notes from my one one one with Elizabeth and will build out more in the next few days

1. Input .csv from experimental team to describe data
2. CSV pulls the proper images from NDP (or wherever best storage location is determined to be)
2a. There needs to be some integration with NDP to break the image into chunks with the appropriate zoom and then provide the original slice with locations of test and training images printed out.
2b. I think to do this we are going to have to register the image to some kind of atlas... or somethign to determine regions so that the code can grab pictures
3. Images need to be split
3a. Regionally
3b. Test Groups
3c. Into Small enough sizes for most efficient processing by package
3d. Magifications
4. Threshold and segment
4a. Ifthresholds (for general object identification) [July]
4b. + Skeletonization (for specific highly branched morphometric analysis) [Robin]
5. Split data into testing and training groups
6. Shape factor analysis on both models (before ML model)
7. Feed 4(1) and 4(2) separately to modified VAMPIRE package to build models
8. Store created models
9. Test models with testing data sets
10. Data output
11. Comprehensive CSV for optional additional analysis
12. Data Visualization
13. Save all data into some storage location (Google Drive?)


Other Notes: 
1. Modified VAMPIRE package: Worth forking the Wirtz lab repository now and building on by ourselves possibly – adding that fork to our repository - COMPLETE
2. Want a repository that works through Binder? Preferably so the Neonatalogy lab can do this straight through a web interface
3. Along with outputs want to output the variables used for all of the steps on days that experiments were run with an easy print out maybe for lab notebook storage? (A way to integrate the electronic lab notebook I want to get more fine tuned for our lab with a standard lab notebook - and provide some under the hood knowledge for Neonatology)

4. Add in an optimization step for what size of image is sufficient for cropping down the whole scan images 
5. Including an analysis before creating that analysis that selects the best # of shape modes and #coordinate points – maybe we could reach out the Denis Wirtz lab about this or build in our own integration



Current Goal:

1. Get a bunch of images from a slice of the ferret brain
2. Put them in a folder
3. Perform thresholding within the Jupyter Notebook
3. Integrate vampire into the Jupyter Notebook
4. Run those images and get an output within the notebook or specific folder (which should will need a results output)
5. Save all of this information to some results folder
6. See if it works in binder

Next Goals:
1. Build out ifthresholds more for immunohistochemistry stains
2. Build in NDP regional registration and automatic image breakdown
3. Integrate Google Drive
4. New visualizations based on what came from paper
5. Statistics possibly with Rthon
6. Speed up any slow processes with Cython
7. Integrate ifThresholds to pick and perform the best thresholding (or integrate a step to say whether this needs to be done or not)

*Step 1: Import necessary packages*

In [98]:
import shutil, os
import re

import numpy as np
import pandas as pd

from skimage import io
from skimage import filters
from skimage import measure
from skimage import morphology
from skimage import segmentation
from skimage import color

%matplotlib inline
import matplotlib.pyplot as plt
#plt.style.use('seaborn-white')

from sklearn.model_selection import train_test_split

from PIL import Image
import image_slicer
from scipy import ndimage as ndi

# I mixed up the dapi and iba so down below it doesn't make sense the final c1 are iba and the final c2s are dapi and I'll fix documentation later

*Step 2: User Inputs*

In [99]:
#Data folder to the stain you are running morphology on (right now only works one stain at a time)
#rerun for each individual stain

#Folder breakdown: stain>animal>region>actual images
data_folder = '/Users/hhelmbre/Desktop/KH_LPS_ferret'

#Image type of your images (they should all be the same type)
image_file_type = '.tif'
file_type_new = '.png'

#Enter the name of your stain as a string
stain1 = 'Iba1' #c1
stain2 = 'dapi' #c2

experiment_name = 'practice'

region_list = ['corpus_callosum', 'hippocampus', 'cortex', 'white_matter', 'basal_ganglia', 'thalamus']
treatment_list = ['NT', 'AcAc', 'OGD1h', 'OGD2h', 'Epo']
female_list = ['68-24', '68-28', '68-35', '68-36', '68-53', '68-60', '44-48', '46-45', '46-47', '64-19', '68-37', '68-43', '68-49', '68-20']
male_list = ['-68-6_', '68-6b', '68-11', '68-12', '68-18', '68-68', '44-42', '46-39', '46-41', '60-1', '64-1', '64-43']


NT_list = ['68-6_', '68-6b', '68-24', '46-39', '46-45', '60-1', '68-20']
AcAc_list = ['68-49', '64-43', '68-53', '68-35', '68-11']
OGD1h_list = ['68-68', '46-41', '46-47', '68-37', '68-28']
OGD2h_list = ['68-36', '44-42', '44-48', '64-1', '64-19', '68-12']
Epo_list = ['68-18', '68-60', '64-7', '68-43']


random_state_num = 1

*Step X: Threshold and segment everything and then split test and train*

In [100]:
folder_list = os.listdir(data_folder)
folder_list = np.asarray(folder_list)

In [101]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for folders in folder_list:
    if mac_annoyance in str(folders):
        folder_list = np.delete(folder_list, (k), axis=0)
    else:
        k+=1

*Step 1: Split up the stains and put them into their own folders*

*Step 1b: Split the images into their individual stains*

In [102]:
for non_channel_split_images in folder_list:
    name = str(data_folder + '/' + non_channel_split_images)
    im = io.imread(name)
    channel1 = im[0, :, :]
    channel2= im[1, :, :]
    filename = non_channel_split_images.replace(image_file_type, "")
    channel1 = Image.fromarray(np.uint16(channel1))
    channel1.save(str(data_folder + '/' + filename + '_' + stain1 + file_type_new))
    channel2 = Image.fromarray(np.uint16(channel2))
    channel2.save(str(data_folder + '/' + filename + '_' + stain2 + file_type_new))
    

*Step 1c: Split the images into 4ths*

In [103]:
arr = os.listdir(data_folder)
file_list = np.asarray(arr)

In [104]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list:
    if mac_annoyance in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

In [105]:
k=0
for images in file_list:
    if '.tif' in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

In [106]:
slice_number = 4
for files in file_list:
    image_slicer.slice(str(data_folder + '/' + files), slice_number)

In [107]:
arr = os.listdir(data_folder)
file_list = np.asarray(arr)

In [108]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list:
    if mac_annoyance in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

*Move split images into their proper stain folders*

In [109]:
os.mkdir(str(data_folder + '/' + stain1))
os.mkdir(str(data_folder + '/' + stain2))

In [110]:
for tiled_images in file_list:
    conditional = str(str(tiled_images)[-6].isdigit())
    if conditional == 'True':
        if stain1 in tiled_images:
            shutil.move(str(data_folder + '/' + tiled_images), str(data_folder + '/' + stain1 + '/' + tiled_images))
        elif stain2 in tiled_images:
            shutil.move(str(data_folder + '/' + tiled_images), str(data_folder + '/' + stain2 + '/' + tiled_images))
    else:
        pass

*Split into the proper sexes*

In [111]:
arr = os.listdir(str(data_folder + '/' + stain1))
file_list_stain1 = np.asarray(arr)

In [112]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list_stain1:
    if mac_annoyance in str(images):
        file_list = np.delete(file_list, (k), axis=0)
    else:
        k+=1

In [113]:
os.mkdir(str(data_folder + '/' + stain1 + '/' + 'female'))
os.mkdir(str(data_folder + '/' + stain1 + '/' + 'male'))

In [114]:
for nonsexed_images in file_list_stain1:
    new_data_folder = data_folder + '/' + stain1 + '/'
    for animal_codes in female_list:
        if animal_codes in nonsexed_images:
            shutil.move(str(new_data_folder + nonsexed_images), str(new_data_folder + 'female/' + nonsexed_images))
        else:
            pass


In [115]:
arr = os.listdir(str(data_folder + '/' + stain1))
file_list_stain1 = np.asarray(arr)

In [116]:
#Code for macs to get rid of an aesthetic file called '.DS_Store'
k=0
mac_annoyance= 'DS_Store'
for images in file_list_stain1:
    if mac_annoyance in str(images):
        file_list_stain1 = np.delete(file_list_stain1, (k), axis=0)
    else:
        k+=1

In [117]:
k = 0
for images in file_list_stain1:
    if '.png' in images:
        k+=1
    else:
        file_list_stain1 = np.delete(file_list_stain1, (k), axis=0)

In [118]:
for nonsexed_images in file_list_stain1:
    new_data_folder = str(data_folder + '/' + stain1 + '/')
    for animal_codes in male_list:
        if animal_codes in nonsexed_images:
            shutil.move(str(new_data_folder + nonsexed_images), str(new_data_folder + 'male/' + nonsexed_images))            
        else:
            pass

*Split into treatment groups*

In [119]:
sexes = ['female', 'male']

In [120]:
for sex in sexes:
    new_data_folder = data_folder + '/' + stain1 + '/' + sex + '/'
    arr = os.listdir(new_data_folder)
    file_list = np.asarray(arr)
    
    k=0
    mac_annoyance= 'DS_Store'
    for images in file_list:
        if mac_annoyance in str(images):
            file_list = np.delete(file_list, (k), axis=0)
        else:
            k+=1
    for treatments in treatment_list:
        os.mkdir(str(new_data_folder + treatments))
        
    
    for sex_split_images in file_list:
        for animal_codes in NT_list:
            if animal_codes in sex_split_images:
                shutil.move(str(new_data_folder + sex_split_images), str(new_data_folder + treatment_list[0] + '/' + sex_split_images))
            else:
                pass
        for animal_codes in AcAc_list:
            if animal_codes in sex_split_images:
                shutil.move(str(new_data_folder + sex_split_images), str(new_data_folder + treatment_list[1] + '/' + sex_split_images))
            else:
                pass
        for animal_codes in OGD1h_list:
            if animal_codes in sex_split_images:
                shutil.move(str(new_data_folder + sex_split_images), str(new_data_folder + treatment_list[2] + '/' + sex_split_images))
            else:
                pass   
        for animal_codes in OGD2h_list:
            if animal_codes in sex_split_images:
                shutil.move(str(new_data_folder + sex_split_images), str(new_data_folder + treatment_list[3] + '/' + sex_split_images))
            else:
                pass 
        for animal_codes in Epo_list:
            if animal_codes in sex_split_images:
                shutil.move(str(new_data_folder + sex_split_images), str(new_data_folder + treatment_list[4] + '/' + sex_split_images))
            else:
                pass 

*Split Regions within each folder*

In [121]:
for sex in sexes:
    for treatments in treatment_list:
        new_data_folder = data_folder + '/' + stain1 + '/' + sex + '/' + treatments + '/'
        arr = os.listdir(str(new_data_folder))
        file_list = np.asarray(arr)
        k=0
        mac_annoyance= 'DS_Store'
        for images in file_list:
            if mac_annoyance in str(images):
                file_list = np.delete(file_list, (k), axis=0)
            else:
                k+=1
        for regions in region_list:
            os.mkdir(str(new_data_folder + regions))
        for stain1_images in file_list:
            region_code = str(stain1_images[12])
            if region_code == '1':
                shutil.move(str(new_data_folder + stain1_images), str(new_data_folder + region_list[0] + '/' + stain1_images))
            elif region_code == '2':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[1] + '/' + stain1_images))
            elif region_code == '3':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[2] + '/' + stain1_images))
            elif region_code == '5':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[3] + '/' + stain1_images))
            elif region_code == '6':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[4] + '/' + stain1_images))
            elif region_code == '7':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[5] + '/' + stain1_images))
            else:
                pass
        for stain1_images in file_list:
            region_code = str(stain1_images[11])
            if region_code == '1':
                shutil.move(str(new_data_folder + stain1_images), str(new_data_folder + region_list[0] + '/' + stain1_images))
            elif region_code == '2':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[1] + '/' + stain1_images))
            elif region_code == '3':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[2] + '/' + stain1_images))
            elif region_code == '5':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[3] + '/' + stain1_images))
            elif region_code == '6':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[4] + '/' + stain1_images))
            elif region_code == '7':
                shutil.move(str(new_data_folder + stain1_images),str(new_data_folder + region_list[5] + '/' + stain1_images))
            else:
                pass

*Step X: Assign test and train groups for each sex>treatment>region group*

In [122]:
test_split_paths = []
for sex in sexes:
    for treatments in treatment_list:
        for regions in region_list:
            path = data_folder + '/' + stain1 + '/' + sex + '/' + treatments + '/' + regions
            test_split_paths.append(path)

In [123]:
os.mkdir(str(data_folder + '/' + 'train'))

In [124]:
for paths in test_split_paths:
    arr = os.listdir(str(paths))
    file_list_train = np.asarray(arr)
    k=0
    mac_annoyance= 'DS_Store'
    for pathways in file_list_train:
        if mac_annoyance in str(images):
            file_list_train = np.delete(file_list_train, (k), axis=0)
        else:
            k+=1
            
    X_train, X_test= train_test_split(file_list_train, test_size=0.25, random_state=random_state_num)
    
    for names in file_list_train:
        if names in X_train[:]:
            shutil.move(str(paths + '/' + names), str(data_folder + '/' + 'train/' + names))
        else:
            pass

*Step X: Grab the DAPI Images for each set and then rename the images*

In [125]:
arr_train1 = os.listdir(str(data_folder + '/' + 'train'))
file_list_train1 = np.asarray(arr_train1)

In [126]:
mac_annoyance= 'DS_Store'
for files in file_list_train1:
    if mac_annoyance in str(files):
        file_list_train1 = np.delete(file_list_train1, (k), axis=0)
    else:
        k+=1

In [127]:
arr_stain2 = os.listdir(str(data_folder + '/' + stain2))
file_list_stain2 = np.asarray(arr_stain2)

In [128]:
for files in file_list_stain2:
    if mac_annoyance in str(files):
        file_list_stain2 = np.delete(file_list_stain2, (k), axis=0)
    else:
        k+=1

In [129]:
im_number= 1
for names in file_list_train1:
    iba_name = str(names)

    if im_number < 10:
        os.rename(str(data_folder + '/' + 'train/' + names), str(data_folder + '/' + 'train/' + 'xy' + '0' + str(im_number) + 'c2.png'))
        
    else:
        os.rename(str(data_folder + '/' + 'train/' + names), str(data_folder + '/' + 'train/' + 'xy' + str(im_number) + 'c2.png'))
    
    dapi_name = iba_name.replace(stain1, stain2)
    
    if im_number < 10:
        os.rename(str(data_folder + '/' + stain2 + '/' + dapi_name), str(data_folder + '/' + 'train/' + 'xy' + '0' + str(im_number) + 'c1.png'))
        
    else:
        os.rename(str(data_folder + '/' + stain2 + '/' + dapi_name), str(data_folder + '/' + 'train/' + 'xy' + str(im_number) + 'c1.png'))
        
    
    im_number +=1

*Grab all the dapi images for the test groups*

In [130]:
for paths in test_split_paths:
    test_stain1 = os.listdir(str(paths))
    test_stain1 = np.asarray(test_stain1)
    for files in test_stain1:
        if mac_annoyance in str(files):
            test_stain1 = np.delete(test_stain1, (k), axis=0)
        else:
            k+=1
    
    im_number= 1
    for names in test_stain1:
        iba_name = str(names)

        if im_number < 10:
            os.rename(str(paths + '/' + names), str(paths + '/' + 'xy' + '0' + str(im_number) + 'c2.png'))

        else:
            os.rename(str(paths + '/' + names), str(paths + '/' + 'train/' + 'xy' + str(im_number) + 'c2.png'))

        dapi_name = iba_name.replace(stain1, stain2)

        if im_number < 10:
            os.rename(str(data_folder + '/' + stain2 + '/' + dapi_name), str(paths + '/' + 'xy' + '0' + str(im_number) + 'c1.png'))

        else:
            os.rename(str(data_folder + '/' + stain2 + '/' + dapi_name), str(paths + '/' + 'train/' + 'xy' + str(im_number) + 'c1.png'))


        im_number +=1

*Step 3: Threshold and Segment Images*

*Step X: Thresholding the training group*

In [131]:
train_list = os.listdir(str(data_folder + '/' + 'train'))
train_list = np.asarray(train_list)

In [132]:
for files in train_list:
    if mac_annoyance in str(files):
        train_list = np.delete(train_list, (k), axis=0)
    else:
        k+=1

*Thresholding on the training files*

In [133]:
os.mkdir(str(data_folder + '/' + 'train/threshold_ims'))
for images in train_list:

    #Creating a folder for the new thresholded images
    #Going through each of the images to get their binarized images and measurement info
    name = str(data_folder + '/' + 'train/' + images)
    im = io.imread(name)
    threshold = filters.threshold_otsu(im)
    binary = morphology.closing(im > threshold, morphology.square(1))
    label_image = measure.label(binary, return_num=False, connectivity=2)
    binary2 = morphology.remove_small_objects(label_image, min_size=25, connectivity=2, in_place=True)
    fill_cells = ndi.binary_fill_holes(binary2)

    # Saving the thresheld images to their own folder with modified names
    im_to_save = Image.fromarray(fill_cells)
    new_name = str(data_folder + '/train/' + 'threshold_ims' + '/' + images)
    new_name = new_name.replace('.png','_threshold.png')
    im_to_save.save(new_name)                

/Users/hhelmbre/opt/anaconda3/envs/vampirenv/lib/python3.7/site-packages/ipykernel_launcher.py:11: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?
  # This is added back by InteractiveShellApp.init_path()


*Thresholding on the Testing Files*

In [134]:
for paths in test_split_paths:
    
    k = 0
    test_stain1 = os.listdir(str(paths))
    test_stain1 = np.asarray(test_stain1)
    for files in test_stain1:
        if mac_annoyance in str(files):
            test_stain1 = np.delete(test_stain1, (k), axis=0)
        else:
            k+=1
    os.mkdir(str(paths + '/' + 'threshold_ims'))
            
    for images in test_stain1:

        #Creating a folder for the new thresholded images
        #Going through each of the images to get their binarized images and measurement info
        name = str(paths + '/' + images)
        im = io.imread(name)
        threshold = filters.threshold_otsu(im)
        binary = morphology.closing(im > threshold, morphology.square(1))
        label_image = measure.label(binary, return_num=False, connectivity=2)
        binary2 = morphology.remove_small_objects(label_image, min_size=25, connectivity=2, in_place=True)
        fill_cells = ndi.binary_fill_holes(binary2)

        # Saving the thresheld images to their own folder with modified names
        im_to_save = Image.fromarray(fill_cells)
        new_name = str(paths + '/threshold_ims' + '/' + images)
        new_name = new_name.replace('.png','_threshold.png')
        im_to_save.save(new_name) 

/Users/hhelmbre/opt/anaconda3/envs/vampirenv/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Only one label was provided to `remove_small_objects`. Did you mean to use a boolean array?


In [7]:
#Go through all the threshold paths + the threshold folder and assign 8 of them for training and 2 of them for testing

# Step 3: Vampire Analysis

*The following will pull up the GUI as a popup - I want to be able to input values into the GUI straight from here not have to point and click*

#We are going to work with what we have an have a print out here of what to input in the VAMPIRE GUI - it is still faster than it used to be. 

(1) Build and then apply model with the vampire GUI (will work on making it not GUI later)

*Step X: Create the CSVs for Building and applying the model*

In [10]:
#create a model repository to store the output information
#create a csv for building the model
#create a csv for applying the model

In [11]:
#creates the directory in your data folder to put all information related to the model
os.mkdir(str(data_folder + '/' + 'model'))

In [12]:
#creates the csv for building a model
data = [['1', stain, str(data_folder + '/' + 'train'), 'ch1', 'iba1']]
build_model_csv = pd.DataFrame(data, columns = ['set ID', 'condition', 'set location', 'tag', 'note']) 

#saves csv to newly created model directory
build_model_csv.to_csv(data_folder + '/' + 'model/' + 'images_to_build_model.csv', index=False)

In [13]:
#creates the csv for applying a model
apply_model_csv = pd.DataFrame(columns = ['set ID', 'condition', 'set location', 'tag', 'note'])

set_number = 1
for folders in test_paths:
    str_set_number = str(set_number)
    pattern = str(data_folder + '/' + '(.*?)' + '/threshold_applied_images/')
    substring = re.search(pattern, folders).group(1)

    df2 = pd.DataFrame({'set ID': [set_number], 'condition': [stain], 'set location': [folders], 'tag': ['ch1'], 'note': ['iba1']})
    apply_model_csv = apply_model_csv.append(df2)
    
    set_number += 1
    
apply_model_csv.to_csv(data_folder + '/' + 'model/' + 'images_to_apply_model.csv', index=False)

Step X: Print the inputs that should be added to VAMPIRE

In [17]:
print('Build Model CSV Path:', str(data_folder + '/' + 'model/' + 'images_to_build_model.csv'))

print('Number of Shape Models (Recommended):', '5')
print('Number of Shape Coordinates (Recommended):', '50')
print('Model Name:', experiment_name)

print('Apply Model CSV Path:', str(data_folder + '/' + 'model/' + 'images_to_apply_model.csv'))
print('Model to Apply:', str(data_folder + '/' + 'model/' + experiment_name))

Build Model CSV Path: /Users/hhelmbre/Desktop/6-17-2020-fiberfimages/iba1/model/images_to_build_model.csv
Number of Shape Models (Recommended): 5
Number of Shape Coordinates (Recommended): 50
Model Name: practice
Model to Apply: /Users/hhelmbre/Desktop/6-17-2020-fiberfimages/iba1/model/practice


*Step X: Open the VAMPIRE GUI, build, and then apply the model*

In [3]:
import vampireanalysis
from vampireanalysis import vampire

In [4]:
vampire()

## getboundary.py


Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/hhelmbre/opt/anaconda3/envs/vampirenv/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2897, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 107, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 131, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1607, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1614, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'ch1'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/hhelmbre/opt/anaconda3/envs/vampirenv/lib/python3.7/tkinter/__init__.py", line 1705, in __call__
    return self.func(*args)
  File "/Users/hhelmbre/opt/anaconda3/envs/vampirenv/lib/python3.7/site-packages/vampireanalysis/vampire.p

*Step 8: New visualization of VAMPIRE data*

- This should probably be built into VAMPIRE and not just into this notebook - think about this for the future

There still needs to be visualization steps here because what is output by vampire just is not great